In [2]:
# Required Python Packages
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from numpy import mean
from collections import Counter
import clogit_estimation
import estimation
import logit
import WhatIfAnalysis

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
%matplotlib inline

In [5]:
# Plotting packages
%matplotlib inline
plt.rcParams['figure.figsize'] = (4, 2)
plt.rcParams['figure.dpi'] = 170
mpl.rc('xtick', labelsize = 6) 
mpl.rc('ytick', labelsize = 6)
plt.rcParams.update({'font.size': 6})

In [6]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [7]:
path = "/Users/frederikluneborgholmjeppesen/Documents/Universitetet/3. år/Bachelorprojektet/MotorRegisterData-main/"
choice_data_subset = pd.read_excel(path + 'choice_data_subset.xlsx')
choice_data_subset['new_prices'] = choice_data_subset['Prices (2015-DKK)']
choice_data_subset

,key,Year,Fuel,Weight (kg),Engine effect (kW),Size (m3),Prices (2015-DKK),Ownership cost (DKK),No. of registrations,Horsepower,Cost/km (DKK),new_prices
0,toyota-avensis-2006-Benzin,2006,Benzin,1820,108.000000,12.573440,403349.299100,346885.205500,5428,139.307692,0.813669,403349.299100
1,volkswagen-passat-2006-Diesel,2006,Diesel,2170,103.000000,12.734904,378504.672900,352803.014000,5378,128.333333,0.681879,378504.672900
2,renault-clio-2006-Benzin,2006,Benzin,1425,43.000000,8.872728,134344.626200,118899.032100,4518,60.000000,0.677246,134344.626200
3,suzuki-swift-2006-Benzin,2006,Benzin,1475,68.000000,9.379500,163536.215000,135059.819400,4406,94.000000,0.734416,163536.215000
4,fiat-punto-2006-Benzin,2006,Benzin,1590,48.000000,9.483248,150287.383200,135188.422500,3743,63.888889,0.689634,150287.383200
...,...,...,...,...,...,...,...,...,...,...,...,...
2785,volkswagen-touareg-2020-Diesel,2020,Diesel,2850,210.000000,16.619328,977674.081200,888497.873100,35,286.000000,0.643713,977674.081200
2786,volkswagen-golf-sportsvan-2020-Benzin,2020,Benzin,1930,110.000000,12.411532,303593.810400,272550.640300,818,150.000000,0.583654,303593.810400
2787,volkswagen-up-2020-El,2020,El,1530,61.000000,8.855775,153689.555100,138192.747300,370,61.222222,0.306765,153689.555100
2788,volkswagen-t-cross-2020-Benzin,2020,Benzin,1760,85.000000,11.429088,263941.972900,237167.811500,2969,115.000000,0.541964,263941.972900


In [8]:
X = choice_data_subset[['Prices (2015-DKK)', 'Weight (kg)', 'Engine effect (kW)', 'Cost/km (DKK)', 'Horsepower']]
X = X.astype(float)

In [9]:
def pris_efter_reg_afgift_ben(afgiftspligtig_værdi, bundfradrag = 21700.0, reg_beloeb1 = 65000.0, reg_beloeb2 = 202200.0,
                                part_1 = 0.25, part_2 = 0.85, part_3 = 1.5):
    CO2_NEDC = 111.6
    CO2_takst = 250
    CO2_tillaeg = CO2_NEDC * CO2_takst
    if afgiftspligtig_værdi < reg_beloeb1:
        reg_part1 = part_1 * afgiftspligtig_værdi
        endelig_reg_afgift = reg_part1 + CO2_tillaeg - bundfradrag # Find ud af om bundfradrag altid skal trækkes fra her
        return afgiftspligtig_værdi + endelig_reg_afgift
    else:
        reg_part1 = part_1 * reg_beloeb1
        if afgiftspligtig_værdi < reg_beloeb2:
            reg_part2 = part_2 * (afgiftspligtig_værdi - reg_beloeb1)
            endelig_reg_afgift = reg_part1 + reg_part2 + CO2_tillaeg - bundfradrag
            return afgiftspligtig_værdi + endelig_reg_afgift
        else:
            reg_part2 = part_2 * (reg_beloeb2 - reg_beloeb1)
            reg_part3 = part_3 * (afgiftspligtig_værdi - reg_beloeb2)
            endelig_reg_afgift = reg_part1 + reg_part2 + reg_part3 + CO2_tillaeg - bundfradrag
            return afgiftspligtig_værdi + endelig_reg_afgift

In [11]:
def pris_efter_reg_afgift_die(afgiftspligtig_værdi, bundfradrag = 21700.0, reg_beloeb1 = 65000.0, reg_beloeb2 = 202200.0,
                                part_1 = 0.25, part_2 = 0.85, part_3 = 1.5):
    CO2_NEDC = 113.9
    CO2_takst = 250
    CO2_tillaeg = CO2_NEDC * CO2_takst
    if afgiftspligtig_værdi < reg_beloeb1:
        reg_part1 = part_1 * afgiftspligtig_værdi
        endelig_reg_afgift = reg_part1 + CO2_tillaeg - bundfradrag # Find ud af om bundfradrag altid skal trækkes fra her
        return afgiftspligtig_værdi + endelig_reg_afgift
    else:
        reg_part1 = part_1 * reg_beloeb1
        if afgiftspligtig_værdi < reg_beloeb2:
            reg_part2 = part_2 * (afgiftspligtig_værdi - reg_beloeb1)
            endelig_reg_afgift = reg_part1 + reg_part2 + CO2_tillaeg - bundfradrag
            return afgiftspligtig_værdi + endelig_reg_afgift
        else:
            reg_part2 = part_2 * (reg_beloeb2 - reg_beloeb1)
            reg_part3 = part_3 * (afgiftspligtig_værdi - reg_beloeb2)
            endelig_reg_afgift = reg_part1 + reg_part2 + reg_part3 + CO2_tillaeg - bundfradrag
            return afgiftspligtig_værdi + endelig_reg_afgift

In [12]:
kWh = 45 #BEMÆRK: Husk at kWh skal indstilles!
def pris_efter_reg_afgift_el(afgiftspligtig_værdi, bundfradrag = 21700.0, ekstra_bundfradrag = 170000.0, indfasning = 0.40, 
                            reg_beloeb1 = 0.25, reg_beloeb2 = 202200.0, el_fradrag = 1700.0, part_1 = 0.25, part_2 = 0.85, part_3 = 1.5):
        samlet_el_fradrag = el_fradrag * kWh
        ny_afgiftspligtig_værdi = afgiftspligtig_værdi - samlet_el_fradrag
        if ny_afgiftspligtig_værdi < reg_beloeb1:
            reg_part1 = part_1 * ny_afgiftspligtig_værdi
            endelig_reg_afgift = reg_part1 - bundfradrag # Find ud af om bundfradrag altid skal trækkes fra her
            return afgiftspligtig_værdi + endelig_reg_afgift
        else:
            reg_part1 = part_1 * reg_beloeb1
            if ny_afgiftspligtig_værdi < reg_beloeb2:
                reg_part2 = part_2 * (ny_afgiftspligtig_værdi - reg_beloeb2)
                reg_før_indfas = reg_part1 + reg_part2 - bundfradrag
                indfasning_reg_afgift = indfasning * reg_før_indfas
                endelig_reg_afgift = indfasning_reg_afgift - ekstra_bundfradrag
                return afgiftspligtig_værdi + endelig_reg_afgift
            else:
                reg_part2 = part_2 * (reg_beloeb2 - reg_beloeb1)
                reg_part3 = part_3 * (ny_afgiftspligtig_værdi - reg_beloeb2)
                reg_før_indfas = reg_part1 + reg_part2 + reg_part3 - bundfradrag
                indfasning_reg_afgift = indfasning * reg_før_indfas
                endelig_reg_afgift = indfasning_reg_afgift - ekstra_bundfradrag
                return afgiftspligtig_værdi + endelig_reg_afgift

In [14]:
goal_seeks = []
for i in range(0, choice_data_subset.shape[0]):
    fuel = choice_data_subset.loc[i, 'Fuel']
    price = choice_data_subset.loc[i, 'Prices (2015-DKK)']
    if fuel == 'El':
        gs = WhatIfAnalysis.GoalSeek(pris_efter_reg_afgift_el, goal = price, x0 = 1000000)
        goal_seeks.append(gs)
    elif fuel == 'Benzin':
        gs = WhatIfAnalysis.GoalSeek(pris_efter_reg_afgift_ben, goal = price, x0 = 1000000)
        goal_seeks.append(gs)
    else:
        gs = WhatIfAnalysis.GoalSeek(pris_efter_reg_afgift_die, goal = price, x0 = 1000000)
        goal_seeks.append(gs)
choice_data_subset = choice_data_subset.assign(afgiftspligtig_værdi = goal_seeks)
#print("Goal seek values:", goal_seeks)
#print(choice_data_subset)

                                        key  Year    Fuel  Weight (kg)  \
0                toyota-avensis-2006-Benzin  2006  Benzin         1820   
1             volkswagen-passat-2006-Diesel  2006  Diesel         2170   
2                  renault-clio-2006-Benzin  2006  Benzin         1425   
3                  suzuki-swift-2006-Benzin  2006  Benzin         1475   
4                    fiat-punto-2006-Benzin  2006  Benzin         1590   
...                                     ...   ...     ...          ...   
2785         volkswagen-touareg-2020-Diesel  2020  Diesel         2850   
2786  volkswagen-golf-sportsvan-2020-Benzin  2020  Benzin         1930   
2787                  volkswagen-up-2020-El  2020      El         1530   
2788         volkswagen-t-cross-2020-Benzin  2020  Benzin         1760   
2789           volkswagen-t-roc-2020-Benzin  2020  Benzin         1860   

      Engine effect (kW)  Size (m3)  Prices (2015-DKK)  Ownership cost (DKK)  \
0             108.000000  12.57

In [15]:
def f(part_1_increase, part_2_increase, part_3_increase, change_tax):
    y = choice_data_subset.loc[:, 'No. of registrations']
    
    X = choice_data_subset[['new_prices', 'Weight (kg)', 'Engine effect (kW)', 'Cost/km (DKK)', 'Horsepower']]
    X = X.astype(float)
        
    if change_tax == True:
        for i in range(0, choice_data_subset.shape[0]):
            if choice_data_subset.Fuel[i] == 'El':
                X.loc[i, 'new_prices'] = pris_efter_reg_afgift_el(afgiftspligtig_værdi = X.loc[i, 'afgiftspligtig_værdi'], 
                                            bundfradrag = 21700.0, ekstra_bundfradrag = 170000.0, indfasning = 0.40, 
                                            reg_beloeb1 = 65000, reg_beloeb2 = 202200.0, el_fradrag = 1700.0, 
                                            part_1 = (1 + part_1_increase) * 0.25, part_2 = (1 + part_2_increase) * 0.85, part_3 = (1 + part_3_increase) * 1.5)

    model = LogisticRegression(multi_class = 'auto', solver = 'lbfgs', tol = 1e-8, max_iter = 10000, penalty = 'l2', C = 1.0).fit(X, y)
    probabilities = model.predict_proba(X)
    
    cp_el = []
    cp_benzin = []
    cp_diesel = []
    
    for i, choice in enumerate(y):
        if choice == 0:
            cp_el.append(probabilities[i, choice])
        
        elif choice == 1:
            cp_benzin.append(probabilities[i, choice])
        
        else:
            cp_diesel.append(probabilities[i, choice])
    
    choice_data_el_subset = choice_data_subset[['Year', 'Fuel']].set_index(['Fuel']).loc[0]
    choice_data_el_subset = choice_data_el_subset.assign(cp_el = cp_el)
    choice_data_el_subset = choice_data_el_subset.groupby('Year')['cp_el'].mean()
    
    choice_data_benzin_subset = choice_data_subset[['Year', 'Fuel']].set_index(['Fuel']).loc[1]
    choice_data_benzin_subset = choice_data_benzin_subset.assign(cp_benzin = cp_benzin)
    choice_data_benzin_subset = choice_data_benzin_subset.groupby('Year')['cp_benzin'].mean()
    
    choice_data_diesel_subset = choice_data_subset[['Year', 'Fuel']].set_index(['Fuel']).loc[2]
    choice_data_diesel_subset = choice_data_diesel_subset.assign(cp_diesel = cp_diesel)
    choice_data_diesel_subset = choice_data_diesel_subset.groupby('Year')['cp_diesel'].mean()
    
    years_el = choice_data_subset[['Year', 'Fuel']].set_index(['Fuel']).loc[0].drop_duplicates()
    years_benzin = choice_data_subset[['Year', 'Fuel']].set_index(['Fuel']).loc[1].drop_duplicates()
    years_diesel = choice_data_subset[['Year', 'Fuel']].set_index(['Fuel']).loc[2].drop_duplicates()

    plt.plot(np.array(years_el), np.array(choice_data_el_subset), c = 'green', label ="El")
    plt.plot(np.array(years_benzin), np.array(choice_data_benzin_subset), c = 'black', label ="Benzin")    
    plt.plot(np.array(years_diesel), np.array(choice_data_diesel_subset), c = 'red', label ="Diesel")
    
    plt.ylim(-0.05, 0.65)
    plt.title('The mean choice probability for each fuel type')
    plt.xlabel("Years")
    plt.ylabel("Mean CP")
    plt.legend(bbox_to_anchor = (1.23, 0.3), ncol = 1)

interact(f, part_1_increase = (0, 1, 0.1), part_2_increase = (0, 1, 0.1), part_3_increase = (0, 1, 0.1), change_tax = False)

interactive(children=(FloatSlider(value=0.0, description='part_1_increase', max=1.0), FloatSlider(value=0.0, d…

<function __main__.f(part_1_increase, part_2_increase, part_3_increase, change_tax)>